# 线性量化

顾名思义，线性量化是将原始浮点数据和量化后的定点数据之间建立一个简单的线性变换关系，因为卷积、全连接等网络层本身只是简单的线性计算，因此线性量化中可以直接用量化后的数据进行直接计算。

浮点和整型之间的换算公式为：
$$r = (q - Z) / scale$$ 
$$q = round(r / S + Z)$$ 

&emsp;&emsp;其中，S 是scale，表示实数和整数之间的比例关系，Z 是 zero point，表示浮点数中的 0 经过量化后对应的整数，它们的计算方法为：
$$S =  \frac{r_{max} - r_{min}}{q_{max} - q_{min}}$$ 
$$Z = round(q_{max}-\frac{r_{max}}{S})$$ 

&emsp;&emsp;其中，$r_{max}$ 和 $r_{max}$分别表示浮点数中的最小值和最大值,$q_{max}$ 和 $q_{min}$分别表示定点数中的最小值和最大值。

![线性量化](../../ch04/images/线性量化.jpg)

## 环境配置

首先，我们安装必须的环境，数据集和model使用和前几章相同的minist数据集和LeNet网络。（PS：最好先创建一个单独的conda环境）

In [ ]:
print('Installing torchprofile...')
# !pip install torchprofile -i https://pypi.tuna.tsinghua.edu.cn/simple/
print('Installing fast-pytorch-kmeans...')
# ! pip install fast-pytorch-kmeans -i https://pypi.tuna.tsinghua.edu.cn/simple/
# ! pip install matplotlib -i https://pypi.tuna.tsinghua.edu.cn/simple/
# ! pip install tqdm -i https://pypi.tuna.tsinghua.edu.cn/simple/
# ! conda install pytorch::pytorch torchvision torchaudio -c pytorch 
# pytorch的安装最好去官网找适合自己的版本命令安装
print('All required packages have been successfully installed!')

Installing torchprofile...
Installing fast-pytorch-kmeans...
All required packages have been successfully installed!


## 构建模型和数据集

数据集和模型的链接如下：

- 模型权重：https://github.com/datawhalechina/awesome-compression/blob/main/docs/notebook/ch02/model.pt
- 数据集：https://github.com/datawhalechina/awesome-compression/tree/main/docs/notebook/ch02/data/mnist/MNIST/raw

创建模型

In [ ]:
# 定义一个LeNet网络
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=16 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=num_classes)

    def forward(self, x):
        x = self.maxpool(F.relu(self.conv1(x)))
        x = self.maxpool(F.relu(self.conv2(x)))

        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet().to(device=device)

数据加载

In [ ]:
# 设置归一化
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# 获取数据集
train_dataset = datasets.MNIST(root='../ch02/data/mnist', train=True, download=True, transform=transform)  
test_dataset = datasets.MNIST(root='../ch02/data/mnist', train=False, download=True, transform=transform)  # train=True训练集，=False测试集

# 设置DataLoader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

加载预训练模型

In [ ]:
# 加载模型的状态字典
checkpoint = torch.load('../ch02/model.pt')
# 加载状态字典到模型
model.load_state_dict(checkpoint)
fp32_model = copy.deepcopy(model)

## 构建训练和验证函数

In [ ]:
def train(
  model: nn.Module,
  dataloader: DataLoader,
  criterion: nn.Module,
  optimizer: Optimizer,
  scheduler: LambdaLR,
  callbacks = None
) -> None:
  model.train()

  for inputs, targets in tqdm(dataloader, desc='train', leave=False):
    # Move the data from CPU to GPU
    # inputs = inputs.to('mps')
    # targets = targets.to('mps')

    # Reset the gradients (from the last iteration)
    optimizer.zero_grad()

    # Forward inference
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Backward propagation
    loss.backward()

    # Update optimizer and LR scheduler
    optimizer.step()
    scheduler.step()

    if callbacks is not None:
        for callback in callbacks:
            callback()

In [ ]:
@torch.inference_mode()
def evaluate(
  model: nn.Module,
  dataloader: DataLoader,
  extra_preprocess = None
) -> float:
  model.eval()

  num_samples = 0
  num_correct = 0

  for inputs, targets in tqdm(dataloader, desc="eval", leave=False):
    # Move the data from CPU to GPU
    # inputs = inputs.to('mps')
    if extra_preprocess is not None:
        for preprocess in extra_preprocess:
            inputs = preprocess(inputs)

    # targets = targets.to('mps')

    # Inference
    outputs = model(inputs)

    # Convert logits to class indices
    outputs = outputs.argmax(dim=1)

    # Update metrics
    num_samples += targets.size(0)
    num_correct += (outputs == targets).sum()

  return (num_correct / num_samples * 100).item()

## 创建两个函数：计算 Flops 和 Model Size

- 参数量（ params ）：
    参数的数量，通常以M为单位。
    params = Kh × Kw × Cin × Cout
- 模型大小(模型大小)：
    在一般的深度学习的框架中（如 PyTorch ），一般是 32 位存储，即一个参数用 32 个 bit 来存储。所以，一个拥有 1M（这里的M是数量单位一百万）参数量的模型所需要的存储空间大小为：1M * 32bit = 32Mb = 4MB。
- 计算量( Flops )：

    即浮点运算数，用来衡量算法/模型的复杂度。图通常只考虑乘加操作的数量，而且只考虑Conv和FC等参数层计算量，忽略BN和PReLU等。一般情况下，Conv和FC层也会忽略仅纯加操作的计算量，如偏置加和shortcut残差加等。目前技术只有BN和CNN可以不加偏置。
    FLOPs = Kh * Kw * Cin * Cout * H * W

In [ ]:
def get_model_flops(model, inputs):
    num_macs = profile_macs(model, inputs)
    return num_macs

def get_model_size(model: nn.Module, data_width=32):
    """
    calculate the model size in bits
    :param data_width: #bits per element
    """
    num_elements = 0
    for param in model.parameters():
        num_elements += param.numel()
    return num_elements * data_width

Byte = 8
KiB = 1024 * Byte
MiB = 1024 * KiB
GiB = 1024 * MiB

## 验证 FP32 模型的精度以及模型大小

In [ ]:
fp32_model_accuracy = evaluate(fp32_model, test_loader)
fp32_model_size = get_model_size(fp32_model)
print(f"fp32 model has accuracy={fp32_model_accuracy:.2f}%")
print(f"fp32 model has size={fp32_model_size/MiB:.2f} MiB")

fp32 model has accuracy=97.99%
fp32 model has size=0.17 MiB
